# Run your first MESA HMS-HMS binary simulation with POSYDON

If you haven't done it already, export all the relevan POSYDON and MESA enviroment variables.

Tip: create a file called `.bash_posydon` with the following content:
```bash
export PATH_TO_POSYDON=/srv/beegfs/scratch/shares/astro/posydon/simone/documentation/POSYDON/
export PATH_TO_POSYDON_DATA=/srv/beegfs/scratch/shares/astro/posydon/POSYDON_GRIDS_v2/POSYDON_data/230914/

export MESA_DIR=/srv/beegfs/scratch/shares/astro/posydon/software/mesa-r11701-posydon-v2-fiximplicit
export OMP_NUM_THREADS=4
export MESASDK_ROOT=/srv/beegfs/scratch/shares/astro/posydon/software/mesasdk-r11701
source $MESASDK_ROOT/bin/mesasdk_init.sh
```
so that every time you open a new terminal you can just run:
```bash
source .bash_posydon
```

If you haven't done it already, please download the `development` branch of the `POSYDON-MESA-INLISTS` submodule. This is the branch that contains the simulation properties of all POSYDON MESA grids.

From the submodule, you can now copy in your working directory the API ini file configuration of HMS-HMS binaries: `$PATH_TO_POSYDON/grid_params/POSYDON-MESA-INLISTS/r11701/running_scripts/HMS-HMS_yggdrasil.ini`. 

For each grid we support two ini file configured to the Northwestern and UNIGE HPC clusters.

In [5]:
import os
import shutil
from posydon.config import PATH_TO_POSYDON

path_to_ini = os.path.join(PATH_TO_POSYDON, "grid_params/POSYDON-MESA-INLISTS/r11701/running_scripts/HMS-HMS_yggdrasil.ini")
shutil.copyfile(path_to_ini, './HMS-HMS_yggdrasil.ini')

'./HMS-HMS_yggdrasil.ini'

Open the file and edit the following parameters:
- `user=your_username`
- `account=your_account`
- `email=your_email`
- `posydon_github_root=your_path_to_POSYDON`

The `scanrio` parameter of the ini file is where the magic happens. This parameter let's us decide which simulation we want to run. The syntax is the following:
- the first argument is the name of the submodule `posydon` which points to the POSYDON-MESA-INLISTS submodule (other options are avaialble, e.g. `user` which points to a private submodule)
- the second argument is the name of the branch and commit of the submodule we want to use connected by a dash `-`. In this case we want to use the `development` branch of the submodule and it's latest commit, so we write `development-c4f90ce2d93595f66751011d2002fc3ab3d090ec`
- the third argument is the name of the simulation grid we want to run, in this case we want to run a HMS-HMS grid. The name of the grid is `HMS-HMS`.

To summarize, you should have `scenario = ['posydon', 'development-c4f90ce2d93595f66751011d2002fc3ab3d090ec', 'HMS-HMS']`

The `zams_file` points to the ZAMS model used to generate the HMS stars in the binary system. POSYDON v2.0.0 supports 8 different metallicities, here we use the default value that points to the 0.1Zsun POSYDON MESA ZAMS model `$PATH_TO_POSYDON/grid_params/POSYDON-MESA-INLISTS/r11701/ZAMS_models/zams_z1.42m3_y0.2511.data`

The last parameter of this file is `grid` and points to the `csv` file containing the initial coditions of the grid. In this case we want to run a `HMS-HMS` grid consisting of one system, so we keep `grid = grid_test.csv` and generate such file specifying the metallicity, the two star masses and the initial orbital period.

Note: a notebook showing you how all the grid points of v2.0.0 are generated is available in the `POSYDON-MESA-INLISTS` submodule at `$PATH_TO_POSYDON/grid_params/POSYDON-MESA-INLISTS/r11701/running_scripts/parameter_space_v2/create_csv.ipynb`

In [7]:
import csv
import numpy as np
import pandas as pd

with open('./grid_test.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['initial_z','Zbase','m1','m2','initial_period_in_days'])
    writer.writerow([0.00142, 0.00142, 30., 21., 10.])

You should now have the following files in your working directory:

In [8]:
!ls

1_hms_hms.ipynb  grid_test.csv	HMS-HMS_yggdrasil.ini


Let's now use the magic of POSYDON to set up the MESA simulation. In your termianal run

```bash
posydon-setup-grid --grid-type fixed --inifile HMS-HMS_yggdrasil.ini --submission-type slurm
```

The following files will be created:

In [11]:
!ls

1_hms_hms.ipynb  grid_test.csv		mk				star1
binary		 HMS-HMS_yggdrasil.ini	slurm_job_array_grid_submit.sh	star2


You are now ready to submit the simulation to the cluster with the following command:

In [12]:
!sbatch slurm_job_array_grid_submit.sh

Submitted batch job 28453658


In [15]:
!squeue -u bavera

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
      28453658_[0] private-a mesa_gri   bavera PD       0:00      1 (Priority)


After the job is finished, we can check the output files:

In [1]:
!ls

1_hms_hms.ipynb
binary
grid_test.csv
HMS-HMS_yggdrasil.ini
mesa_grid.28453658_0.out
mk
slurm_job_array_grid_submit.sh
star1
star2
Zbase_0.0014_m1_30.0000_m2_21.0000_initial_z_1.4200e-03_initial_period_in_days_1.0000e+01_grid_index_0


In [2]:
!ls Zbase_0.0014_m1_30.0000_m2_21.0000_initial_z_1.4200e-03_initial_period_in_days_1.0000e+01_grid_index_0

binary_history_columns.list  inlist_grid_points
binary_history.data	     inlist_grid_star1_binary_controls
final_star1.mod		     inlist_grid_star2_binary_controls
final_star2.mod		     LOGS1
history_columns.list	     LOGS2
initial_star1.mod	     out.txt
initial_star2.mod	     profile_columns.list
inlist			     tmp.hdf5


In [4]:
!head -n 200 Zbase_0.0014_m1_30.0000_m2_21.0000_initial_z_1.4200e-03_initial_period_in_days_1.0000e+01_grid_index_0/out.txt



 read /srv/beegfs/scratch/shares/astro/posydon/simone/documentation/running_mesa/running_1_hms_hms/binary/inlist_project
 read inlist_grid_points
                                         version_number       11701
 read inlist_grid_star1_binary_controls
 set_eos_PC_parameters
                             mass_fraction_limit_for_PC    1.0000000000000000D-03
                                       logRho1_PC_limit    2.9990000000000001D+00
                                       logRho2_PC_limit    2.7999999999999998D+00
                                     log_Gamma_all_HELM    1.0000000000000000D+00
                                       log_Gamma_all_PC    1.3010299956000000D+00
                                 PC_Gamma_start_crystal    1.5000000000000000D+02
                                  PC_Gamma_full_crystal    1.7500000000000000D+02
                                               PC_min_Z    9.9900000000000000D-01
 change rates preference to           2
                         

Congratulation, you now know how to harvast the power of POSYDON to run a grid of simulations on a supercomputer!